In [1]:
#Tässä notebookissa analysoidaan tiedosto, jossa on lueteltu henkilön kuukausipalkka ja ikä. Data on tekoälyn
#tuottamaa. Analysoidaan palkkadata ja lopuksi tutkitaan, että korreloiko palkansaajan ikä palkan suuruuden kanssa.  

#ladataan aluksi data Excelistä dataframeen. Argumentti infer_eltypesin avulla
#yritetään päätellä sarakkeiden tietotyypit automaattisesti. 

using DataFrames, XLSX

data = DataFrame(XLSX.readtable("C:\\Users\\Installer\\Desktop\\palkatjaika.xlsx","data", infer_eltypes=true))


Row,Ika,Kuukausipalkka
,Int64,Int64
1,49,4929
2,42,3759
3,51,8348
4,63,11400
5,45,4712
6,46,2813
7,49,5728
8,45,3331
9,44,4747


In [2]:
#Määritetään seuraavaksi palkkojen tunnuslukuja komennolla describe. Jokaiselle tunnusluvulle 
#on myös olemassa erillinen komento, jota voidaan tarvittaessa käyttää. 

describe(select(data, :Kuukausipalkka)) 

#describe ei toimi vektoreille, joten tarvitaan select-komentoa, joka palauttaa dataframen.



Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,DataType
1,Kuukausipalkka,4083.83,1530,3500.0,12000,0,Int64


In [3]:
#describe ei anna desiilejä. Määritetään ne seuraavaksi. Komento tulostaa 11 arvoa, minimin ja desiilit 1-10. 
#Desiilithän jakavat järjestetyn datan kymmeneen yhtä suureen osaan. #Huomaa, että viides desiili on mediaani ja 
#kymmenes desiili on maksimi. 

using Statistics

quantile(data.Kuukausipalkka, 0:0.1:1)





11-element Vector{Float64}:
  1530.0
  1923.8
  2317.6000000000004
  2712.4
  3106.2000000000003
  3500.0
  4000.2000000000003
  4499.9
  5000.8
  7001.399999999998
 12000.0

In [4]:
#Tehdään seuraavaksi frekvenssitaulukko palkoista siten, että jaetaan palkat luokkiin 1000 € välein. 
#Ensimmäinen luokka on vähemmän kuin 2000 €, seuraava 2001 - 3000 €, jne. Viimeinen luokka on 
#11001 - 12000 €. 

#Muista asentaa paketti Statsbase. 

using StatsBase

# Luokkarajat. Alaraja on inklusiivinen ja yläraja eksklusiivinen. Näin ollen esim. palkka 2000 kuuluu luokkaan 2000 - 3000.
luokat = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000]

# Tehdään Histogram-olio annetuilla rajoilla. Olio sisältää luokkarajat (h.edges) ja frekvenssit (h.weights). 
h = fit(Histogram, data.Kuukausipalkka, luokat)

# Tulostetaan frekvenssit taulukkona
print(h.edges)
print(h.weights)


([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000],)[239, 507, 454, 400, 100, 100, 40, 40, 40, 40, 39]

In [5]:
#Tarkistetaan tulivatko kaikki arvot mukaan. 

sum(h.weights)

1999

In [6]:
#Huomataan, että yksi arvo on jäänyt pois. Tämä johtuu siitä, että yläraja on eksklusiivinen ja arvo 12000 jää pois. 
#Muokataan viimeistä ylärajaa ja tarkistetaan tulos. 

# Luokkarajat. Alaraja on inklusiivinen ja yläraja eksklusiivinen. Näin ollen esim. palkka 2000 kuuluu luokkaan 2000 - 3000. 
#Arvo 12 000 kuuluu kuitenkin viimeiseen luokkaan, koska sille annetaan ylärajaksi 12001. 

luokat = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12001] #luodaan vektori

# Tehdään Histogram-olio annetuilla rajoilla. Olio sisältää luokkarajat (h.edges) ja frekvenssit (h.weights). 
h = fit(Histogram, data.Kuukausipalkka, luokat)

print("$(sum(h.weights))\n")
print(h.edges)
print(h.weights)

2000
([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12001],)[239, 507, 454, 400, 100, 100, 40, 40, 40, 40, 40]

In [7]:
#Huomataan, että tulostus ei kerro ihmissilmälle juuri mitään. Muodostetaan siis kunnollinen frekvenssitaulukko. 
#Tarkoituksena on tehdä frekvenssitaulukko, jossa on neljä erilaista frekvenssiä. Luodaan siis dataframe, jonka 
#ensimmäiseksi sarakkeeksi annetaan luokat ja toiseksi frekvenssit. Tehdään aluksi molemmista vektorit. Olisi myös
#mahdollista lisätä ne suoraan dataframeen. 

luokkarajat = String[] #tyhjä string-tyyppinen vektori

#Kiinnitä huomiota Julian tapaan yhdistää muuttujien arvoja ja tekstiä string-tyyppiseksi muuttujaks. $(dollarimerkki eteen ja sulut).

for i in 1:length(luokat)-1 #Mieti, miksi tässä pitää olla -1. Voit kokeilla ottaa sen pois. 
    push!(luokkarajat, 
            "$(luokat[i]) – $(luokat[i+1]-1)"
        )
end

luokkarajat



11-element Vector{String}:
 "1000 – 1999"
 "2000 – 2999"
 "3000 – 3999"
 "4000 – 4999"
 "5000 – 5999"
 "6000 – 6999"
 "7000 – 7999"
 "8000 – 8999"
 "9000 – 9999"
 "10000 – 10999"
 "11000 – 12000"

In [8]:
#Luodaan dataframe ja annetaan sille luokkarajat ensimmäiseen sarakkeeseen.

Fr_taul = DataFrame(
    Palkka = luokkarajat
    )


Row,Palkka
,String
1,1000 – 1999
2,2000 – 2999
3,3000 – 3999
4,4000 – 4999
5,5000 – 5999
6,6000 – 6999
7,7000 – 7999
8,8000 – 8999
9,9000 – 9999


In [9]:
#Lisätään frekvenssit dataframeen. 

Fr_taul[!, :Frekvenssi] = h.weights

Fr_taul



Row,Palkka,Frekvenssi
,String,Int64
1,1000 – 1999,239
2,2000 – 2999,507
3,3000 – 3999,454
4,4000 – 4999,400
5,5000 – 5999,100
6,6000 – 6999,100
7,7000 – 7999,40
8,8000 – 8999,40
9,9000 – 9999,40


In [10]:
#Lisätään kumulatiiviset frekvenssi dataframeen.

kum_fr = cumsum(h.weights)

Fr_taul[!, :Kumulatiivinen_fr] = kum_fr

Fr_taul


Row,Palkka,Frekvenssi,Kumulatiivinen_fr
,String,Int64,Int64
1,1000 – 1999,239,239
2,2000 – 2999,507,746
3,3000 – 3999,454,1200
4,4000 – 4999,400,1600
5,5000 – 5999,100,1700
6,6000 – 6999,100,1800
7,7000 – 7999,40,1840
8,8000 – 8999,40,1880
9,9000 – 9999,40,1920


In [11]:
#Määritetään seuraavaksi suhteelliset frekvenssit ja lisätään ne dataframeen. 

suht_fr = (h.weights ./ sum(h.weights))*100 #Muutetaan arvo prosentiksi ja lisätään perään %. 
#Pisteellä (.) kerrotaan, että jakolasku tehdään vektorin jokaiselle alkiolle erikseen.

Fr_taul[!, "Suhteellinen_fr_(%)"] = suht_fr 

#Nyt sarakeotsikon tietotyyppi on string, kun se normaalisti on symbol. Tämä saattaa aiheuttaa ongelmia. 

Fr_taul


Row,Palkka,Frekvenssi,Kumulatiivinen_fr,Suhteellinen_fr_(%)
,String,Int64,Int64,Float64
1,1000 – 1999,239,239,11.95
2,2000 – 2999,507,746,25.35
3,3000 – 3999,454,1200,22.7
4,4000 – 4999,400,1600,20.0
5,5000 – 5999,100,1700,5.0
6,6000 – 6999,100,1800,5.0
7,7000 – 7999,40,1840,2.0
8,8000 – 8999,40,1880,2.0
9,9000 – 9999,40,1920,2.0


In [12]:
#Määritetään vielä suhteellinen kumulatiivinen frekvenssi ja lisätään se dataframeen.

        
kum_suht = cumsum(suht_fr)                     


Fr_taul[!, "Kumulatiivinen_suhteellinen_fr_(%)"] = kum_suht

Fr_taul



Row,Palkka,Frekvenssi,Kumulatiivinen_fr,Suhteellinen_fr_(%),Kumulatiivinen_suhteellinen_fr_(%)
,String,Int64,Int64,Float64,Float64
1,1000 – 1999,239,239,11.95,11.95
2,2000 – 2999,507,746,25.35,37.3
3,3000 – 3999,454,1200,22.7,60.0
4,4000 – 4999,400,1600,20.0,80.0
5,5000 – 5999,100,1700,5.0,85.0
6,6000 – 6999,100,1800,5.0,90.0
7,7000 – 7999,40,1840,2.0,92.0
8,8000 – 8999,40,1880,2.0,94.0
9,9000 – 9999,40,1920,2.0,96.0


In [13]:
#Tarkistetaan lopuksi, että korreloiko palkansaajan ikä palkan määrän kanssa. Käytetään Pearsonia. 

using Statistics

cor(data.Ika, data.Kuukausipalkka)

0.6976663452760882

In [14]:
#Nyt korrelaatio näyttäisi olevan kohtuullinen, mutta jotta johtopäätöksiä voidaan tehdä, täytyy 
#laskea myös p-arvo. Tähän käytetään CorrelationTest-funktiota kirjasto HypothesisTests. 
#Asenna kirjasto tarvittaessa ennen koodin kokeilemista. 

#Tehdään hypoteesit ennen testiä: H0: Ikä ja palkka eivät korreloi. H1: Ikä ja palkka korreloivat. 
#Käytetään 95 % merkitsevyystasoa. 

using HypothesisTests

testi = CorrelationTest(data.Ika, data.Kuukausipalkka)

#Mieti, mitä tulosteessa tarkoittaa 95 % luottamusväli!


Test for nonzero correlation
----------------------------
Population details:
    parameter of interest:   Correlation
    value under h_0:         0.0
    point estimate:          0.697666
    95% confidence interval: (0.6745, 0.7195)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-99

Details:
    number of observations:          2000
    number of conditional variables: 0
    t-statistic:                     43.5287
    degrees of freedom:              1998


In [15]:
#p-arvo näytetään 10 potensseina. Tulostetaan se ja Pearsonin korrelaatiokertoimen arvot 
#erikseen viiden desimaalin tarkkuudella. 

using Printf

p = pvalue(testi)

@printf("r = %.5f, p = %.5f\n", testi.r, p)


r = 0.69767, p = 0.00000


In [16]:
#Nyt on 0,00000 % virhemahdollisuus, jos hylkäämme nollahypoteesin. Koska 95 % merkitsevyystasolla hyväksytään
#5 % virhemahdollisuus, nollahypoteesi hylätään vain vaihtoehtoinen hypoteesi jää voimaan -->
#ikä ja palkka siis korreloivat. 

#Lasketaan lopuksi korrelaatiokertoimen selitysaste.
print("Selitysaste on $(round(100 * testi.r^2; digits = 1)) %")


Selitysaste on 48.7 %

In [17]:
#Selitysaste on siis 48.7 %. Tämä tarkoittaa sitä, että 48,7 % palkan vaihtelusta voidaan
#selittää iän vaihtelulla. 